# KINTSUGI

## In the following notebook you will prepare for processing your images; test illumination correction parameters, stitching accuracy, and deconvolution.

## 1. Import packages. This must be done every time the notebook is started or restarted.

Import these packages.  Run cells using Ctrl+Enter.

In [1]:
import concurrent.futures
import gc
import os
import tkinter as tk
import errno
from tkinter import filedialog
from tkinter import simpledialog
import pandas as pd
from m2stitch import stitch_images
from basicpy import BaSiC, metrics
from glob import glob
from skimage.io.collection import alphanumeric_key
import shutil
from hyperactive import Hyperactive
import numpy as np
from skimage.io import imread 
from skimage.io import imsave
from skimage import io
from skimage import exposure
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import jax
import pickle
from itertools import chain, repeat
import subprocess
import imagej, scyjava

jax.config.update('jax_platform_name', 'cpu')

## 2. Define directory paths.  *This must be done every time the notebook is started or restarted.

If you are familiar with defining directories, enter them below.  If not the other cells will assist with the necessary definitions.

Below are two ways to get the required paths to input, output, and meta folders.  The first is where they can be entered manually.  The second assists with the process.  

Choose only one method: A or B

### 2.1 Method A

In [2]:
image_dir ="C:/Users/smith6jt/1904_CC2B28_raw"
stitch_dir = "C:/Users/smith6jt/1904_CC2B28_BaSiC_Stitched"
meta_dir = "C:/Users/smith6jt/1904_CC2B28_meta"
print(f"Image folder is {image_dir}.")
print(f"Stitching folder is {stitch_dir}.")
print(f"Meta folder is {meta_dir}.")

Image folder is C:/Users/smith6jt/1904_CC2B28_raw.
Stitching folder is C:/Users/smith6jt/1904_CC2B28_BaSiC_Stitched.
Meta folder is C:/Users/smith6jt/1904_CC2B28_meta.


### 2.1 Method B

First, assess the folder and file names.  For example, in CODEX output, the original folder name is something like CX_20-008_SP_CC2-B. We can shorten that now to make life easier down the road.  Shorten the folder name to remove redundant information.  In this example we choose "2008_CC2B_raw" making sure the '_raw' is the last part.

Running the following cell will bring up a dialog window where you will enter the new folder name and select the folder containing your images.  This folder will be renamed designated as the image_dir for the project.  An stitching output folder and a folder for metadata files will also be created.  Make sure there are only a series of folders each containing the images for each cycle in the folder you select.

In [ ]:
root = tk.Tk()
root.withdraw()

user_choice = simpledialog.askstring("Shortened name", "Enter shortened file name with _raw at end")
image_dir_long = filedialog.askdirectory()
head_dir, tail_dir = os.path.split(image_dir_long)

try:  
    os.rename(image_dir_long, os.path.join(head_dir, user_choice))
except FileNotFoundError:
    print("The file or directory does not exist.")
except PermissionError:
    print("you don't have permissions to rename the file")
except OSError as error:
    print(f"Error: {error}")

if os.path.isdir(os.path.join(head_dir, user_choice)) == True:
    image_dir = os.path.join(head_dir, user_choice)
    print(f"{image_dir_long} renamed to {image_dir}")

else:
    print("Error in renaming.")
    
stitch_dir = image_dir.replace('_raw', '_BaSiC_Stitched')
meta_dir = image_dir.replace('_raw', '_meta')
os.makedirs(stitch_dir, exist_ok=True)
os.makedirs(meta_dir, exist_ok=True)

if os.path.isdir(stitch_dir) == True:
    print(f"Stitching output folder: {stitch_dir} created successfully.")

else:
    print("Stitching output folder not created.")

if os.path.isdir(meta_dir) == True:
    print(f"Metadata folder: {meta_dir} created successfully.  Move metadata files to metadata folder.")

else:
    print("Metadata folder not created.")

Optional. Save these variables in a text file to use later.

In [ ]:
project_file = os.path.join(meta_dir, "project_data.txt")
variables = globals()
variable_names_to_save = ['stitch_dir', 'meta_dir', 'image_dir']

with open(project_file, 'w') as file:
    for var_name in variable_names_to_save:
        if var_name in variables:
            var_value = variables[var_name]
            var_str = f'{var_name}={var_value}\n'
            file.write(var_str)

### 2.3 Shorten cycle folder names

NOTE: Running this cell permanently alters the cycle folder names.  This only needs to be run once.

If the cycle folders have long names starting with the cycle numbers followed by a split character (e.g. the underscore in 'cyc001_reg001_200210_170925') this cell will shorten the folder name to everything before the first split character.

Make sure to run the codeblock above before moving on to the next code block.

Enter the split character in quotations.  It is usually an underscore.

Note that the rest of the folder name will be deleted unless you add additional arguments to the os.rename function.

In [ ]:
# Rename long source folder names

split_character = simpledialog.askstring("Split character", "Enter split character")
print(f'Split character is {split_character}.')
if split_character == 'None':
    print('No input given.')

else:
    for cyc_folder in os.listdir(image_dir):
        new_name = cyc_folder.split(split_character)
        cycle_dir = os.path.join(image_dir, cyc_folder)
        cycle_dir_short = os.path.join(image_dir, new_name[0])
        new_cycle_name = os.rename(cycle_dir, cycle_dir_short)
        if os.path.isdir(new_cycle_name) == True:
            print(f"{cycle_dir} renamed to {cycle_dir_short}")

## 3. Testing functions

Before running very large image datasets, first inspect a sample of images, and then check the quality of illumination correction, stitching, and deconvolution.  Here parameters can be tested, tuned, and refined as necessary with visual inspection of a single cycle, zplane, channel combination.  Starting with the nuclear staining channel or other channels necessary for segmentation is a good idea, then make sure to assess channels with differing staining patterns and intensities.

Make sure the directories are defined!

The zplane and channel numbers of interest, and the pattern of the image_file name must be entered below.  The wildcard characters "??" are entered where the image tile numbers are so that all tiles are loaded.

The cycle folder names are assumed to have been shortened and in the format "cyc00x".  Running the cell will prompt to enter the cycle folder of interest.
    

In [3]:
cycle_folder = filedialog.askdirectory()

zplane = 8
channel = 1

# image_file name below is derived from 1_000tt_Z0zz_CHc.tif where tt is the tile number with one leading zero, zz is the z-position with one leading zero, and c is the channel number.
image_file = f'1_000??_Z0{str(zplane).zfill(2)}_CH{channel}.tif'
print(f"Using {cycle_folder}/{image_file}")

#Convert to numpy array
im_raw = sorted(glob(os.path.join(cycle_folder, image_file)), key = alphanumeric_key)
im = io.imread_collection(im_raw)
im_array = np.asarray(im)
print(f"Type: {im_array.dtype}; Min pixel: {str(np.min(im_array))}; Max pixel: {str(np.max(im_array))}")

if np.min(im_array) != 0:
    im_array = im_array - np.min(im_array)

np_mask = np.isnan(im_array)
nan_indices = np.where(np_mask)[0]

print(f"Set of bad pixels: {nan_indices}. Check if not []") 

print(f"Min pixel: {str(np.min(im_array))} Max pixel: {str(np.max(im_array))}")

Using C:/Users/smith6jt/1904_CC2B28_raw/cyc001/1_000??_Z008_CH1.tif
Type: uint16; Min pixel: 0; Max pixel: 32121
Set of bad pixels: []. Check if not []
Min pixel: 0 Max pixel: 32121


### 3.1 Illumination Correction

For illumination correction, we use the BaSiCPy package.  An important consideration is whether or not to calculate darkfield.  There are many more arguments for the BaSiC and autotune functions that can be manipulated as needed.  For more information see: https://github.com/peng-lab/BaSiCPy/tree/main.

To reuse a set of parameters, uncomment the load_model and save_model lines while commenting out the first two lines.  

In [4]:

from skimage.filters import sobel
from scipy.stats import iqr

# Function to calculate parameter ranges for a single image
def calculate_parameters(image):
    # Smoothness
    gradient_variance = np.var(sobel(image))
    mu_range = [0.001, 1.0 / max(gradient_variance, 1e-3)]

    # Flat field smoothness
    intensity_variance = np.var(image)
    lambda_flat_range = [0.1, min(10, 1.0 / max(intensity_variance, 1e-3))]

    # Dark field
    lower_quantile = np.percentile(image, 1)
    alpha_range = [0, lower_quantile * 2]

    # Sparse dark field
    noise_level = np.std(image[image < lower_quantile])
    lambda_sparse_range = [0.1, 1.0 / max(noise_level, 1e-3)]

    return mu_range, lambda_flat_range, alpha_range, lambda_sparse_range

# Aggregate ranges for the dataset
def aggregate_ranges(dataset):
    mu_all, flat_all, alpha_all, sparse_all = [], [], [], []

    for image_path in dataset:
        image = io.imread(image_path)
        image = image / np.max(image)  # Normalize to 0-1
        mu, flat, alpha, sparse = calculate_parameters(image)

        mu_all.append(mu)
        flat_all.append(flat)
        alpha_all.append(alpha)
        sparse_all.append(sparse)

    # Use robust statistics (e.g., 5%-95% range)
    mu_range = [np.min(mu_all), np.percentile(mu_all, 95)]
    flat_range = [np.min(flat_all), np.percentile(flat_all, 95)]
    alpha_range = [np.min(alpha_all), np.percentile(alpha_all, 95)]
    sparse_range = [np.min(sparse_all), np.percentile(sparse_all, 95)]

    return mu_range, flat_range, alpha_range, sparse_range

# Example usage
dataset = im_raw
parameter_ranges = aggregate_ranges(dataset)
print("Parameter Ranges:", parameter_ranges)


Parameter Ranges: ([0.001, 1000.0], [0.1, 10.0], [0.0, 0.07088952033878887], [0.1, 406.1394851597288])


View the correction profiles.

In [ ]:
project_file = os.path.join(meta_dir, "project_data.txt")
variables = globals()
variable_names_to_save = ['stitch_dir', 'meta_dir', 'image_dir']

with open(project_file, 'w') as file:
    for var_name in variable_names_to_save:
        if var_name in variables:
            var_value = variables[var_name]
            var_str = f'{var_name}={var_value}\n'
            file.write(var_str)

In [ ]:
mu_values = []
lam_values = []
alpha_values = []
lam_sparse = []

for im in im_raw:
    # Example for a single image
    image = io.imread(im)

    # Smoothness
    gradient_variance = np.var(sobel(image))
    mu_range = [0.001, 1.0 / max(gradient_variance, 1e-3)]
    

    # Flat field smoothness
    intensity_variance = np.var(image)
    lambda_flat_range = [0.1, min(10, 1.0 / max(intensity_variance, 1e-3))]

    # Dark field
    lower_quantile = np.percentile(image, 1)
    alpha_range = [0, lower_quantile * 2]

    # Sparse dark field
    noise_level = np.std(image[image < lower_quantile])  # Assume low-intensity pixels are noise
    lambda_sparse_range = [0.1, 1.0 / max(noise_level, 1e-3)]


## 3.2 Stitching

Here we first stitch the original uncorrected tiles followed by the corrected tiles.  This is accomplished using the package m2stitch.  Enter the number of rows and columns, and overlap percentage known from your data.  The "pou" stands for percent overlap uncertainty and is the sole parameter that can be changed to affect stitching quality.  The rows and columns functions are for generating lists corresponding to a "snake by rows" pattern starting at the upper left of the image.  They will need to be changed for other stitching patterns.

To save and reuse stitching results, uncomment/comment out the appropriate lines.  The results are saved to the metadata folder defined above.

Be sure to closely evaluate the areas of overlap for stitching quality.  The cells below allow for visualizing in this notebook and for saving to inspect elsewhere.

For more information see https://github.com/yfukai/m2stitch

In [ ]:
n = 11  # Number of rows (height)
m = 9 # Number of columns (width)
overlap_percentage = 0.30
pou = 12
stitch_model = os.path.join(meta_dir, "result.pkl")

# Row coordinates: each row index is repeated m times
rows = list(chain.from_iterable(repeat(row, m) for row in range(n)))

# Column coordinates: snake pattern for each row, going back and forth
cols = list(chain.from_iterable(
    range(m) if row % 2 == 0 else range(m - 1, -1, -1) for row in range(n)
))
result_df, _ = stitch_images(im_array, rows, cols, initial_ncc_threshold = 0, overlap_percentage=overlap_percentage, pou=pou)
result_df.to_pickle(stitch_model)

If the cell above has already been run to create a stitching model, run the cell below to apply it to the original images.

In [ ]:
stitch_model = os.path.join(meta_dir, "result.pkl")

result_df=pd.read_pickle(stitch_model)

result_df["y_pos2"] = result_df["y_pos"] - result_df["y_pos"].min()
result_df["x_pos2"] = result_df["x_pos"] - result_df["x_pos"].min()

size_y = im_array.shape[1]
size_x = im_array.shape[2]

stitched_image_size = (
    result_df["y_pos2"].max() + size_y,
    result_df["x_pos2"].max() + size_x,
)
stitched_image = np.zeros_like(im_array, shape=stitched_image_size)
for i, row in result_df.iterrows():
    stitched_image[
        row["y_pos2"] : row["y_pos2"] + size_y,
        row["x_pos2"] : row["x_pos2"] + size_x,
    ] = im_array[i]

Visualize the result of stitching the original tiles.

In [ ]:
stitched_image_re = exposure.rescale_intensity(stitched_image, in_range=(np.min(stitched_image), np.max(stitched_image)), out_range=(0, 65535)).astype(np.uint16)
fig, axes = plt.subplots(figsize=(24, 18))
im = axes.imshow(stitched_image_re, vmax=55000)
fig.colorbar(im)
fig.tight_layout()

Save the stitched image if desired.

In [ ]:
stitched_image_2 = exposure.rescale_intensity(stitched_image, in_range=(np.min(stitched_image), np.max(stitched_image)), out_range=(0, 65535)).astype(np.uint16)
raw_image_file_path = os.path.join(meta_dir, "test_original.tif") 
imsave(raw_image_file_path, stitched_image_2)

Reuse the stitching model to stitch the corrected tiles.

In [ ]:
stitch_model = os.path.join(meta_dir, "result.pkl")
result_df=pd.read_pickle(stitch_model)
result_df["y_pos2"] = result_df["y_pos"] - result_df["y_pos"].min()
result_df["x_pos2"] = result_df["x_pos"] - result_df["x_pos"].min()

size_y = images_transformed.shape[1]
size_x = images_transformed.shape[2]

stitched_image_size = (
    result_df["y_pos2"].max() + size_y,
    result_df["x_pos2"].max() + size_x,
)
stitched_image_basic = np.zeros_like(images_transformed, shape=stitched_image_size)
for i, row in result_df.iterrows():
    stitched_image_basic[
        row["y_pos2"] : row["y_pos2"] + size_y,
        row["x_pos2"] : row["x_pos2"] + size_x,
    ] = images_transformed[i]

Visualize the result of stitching the corrected tiles.  Comment/uncomment the coordinates to crop.

In [ ]:
stitched_image_basic_re = exposure.rescale_intensity(stitched_image_basic, in_range=(np.min(stitched_image_basic), np.max(stitched_image_basic)), out_range=(0, 65535)).astype(np.uint16)
# x1 = 6000
# x2 = 8000
# y1 = 0000
# y2 = 3000
fig, axes = plt.subplots(figsize=(24, 18))
# im = axes.imshow(stitched_image_basic[y1:y2,x1:x2])
im = axes.imshow(stitched_image_basic_re, vmax=55000)
fig.colorbar(im)
fig.tight_layout()

Save the corrected stitched image if desired.

In [ ]:
stitched_image_basic_re = exposure.rescale_intensity(stitched_image_basic, in_range=(np.min(stitched_image_basic), np.max(stitched_image_basic)), out_range=(0, 65535)).astype(np.uint16)
result_image_file_path = os.path.join(meta_dir, "test_corrected.tif") 
imsave(result_image_file_path, stitched_image_basic_re, check_contrast=False)

Before testing deconvolution, you must correct and stitch an entire z-stack.  For this you can run the cells above for each image in the stack, or use B_IllumCor_Stitching_Decon.ipynb

## 3.3 Deconvolution

The following cell has multiple parameters related to the microscope used to aquire the images and the images themselves.  Once these are input, the testing is primarily to tune the number of iterations/stop_crit necessary to get good results.  The damping and hist_clip further can improve results.  There is no need to supply a point-spread function.  Finally, finding the optimal use of computing resources on GPU or CPU is accomplished via the max_GPU and max_CPU parameters.  There is no limit to image size.  If the images are too large to fit in the specified memory maximums, the will be split into blocks and processed sequentially.

Be sure the MATLAB Runtime v9.5 (R2018, 64-bit) is installed.  Download for free here: http://www.mathworks.com/products/compiler/mcr/index.html. Reboot your computer after install.  

The image_dir and stitch_dir definitions used previously will be again used here, and a new folder will be created to save the deconvolved images.  

The original deconvolution program was for lightsheet images and modified to use with widefield fluorescence.  See https://www.nature.com/articles/s41598-019-53875-y 

Output will be written to the terminal.  Check that the first image stack is processed without error.  If there is an "Maximum variable size allowed on the device is exceeded." error, then restart the kernel, decrease the max_GPU or max_CPU parameter, rerun the decon function cell, and try again.

In [ ]:
dec_cycles = 1
dec_channels = 1

# pixel size in xy dimension (nanometers)
xy_vox = 377
# pixel size in z dimension (nanometers)
z_vox = 1500
# Number of iterations of Lucy-Richardson algo before stopping unless stop_crit is met first
iterations = 25
# Microscope objective numerical aperture
mic_NA = 0.75
# Refractive index of tissue being imaged
tissue_RI = 1.3
# Opening size in millimeters of objective aperture
slit_aper = 6.5
# Focal length in millimeters of objective
f_cyl = 1
# Used to reduce noise.  Increase value for noisy images. (0-10)
damping = 0
# If set, the deconvolved images will be clipped by this percent for max and min values, and then scaled to full range of bit depth. (0-5)
hist_clip = 0.010
# Percent change between iterations to use as criteria to stop deconvolution.
stop_crit = 5.00
# Enter 1 to perform on GPU, 0 to use CPU
GPU = 1
# Percent maximum GPU memory to use if GPU = 1
max_GPU = 45
# Percent maximum RAM to use if GPU = 0
max_CPU = 20
if GPU == 1:
    max_block=max_GPU
elif GPU == 0:
    max_block=max_CPU
# The excitation and emission wavelength in nanometers
ex = 358
em = 461

decon_exe = os.path.join(os.path.dirname(image_dir), "LsDeconv.exe")
decon_dir = stitch_dir.replace('_BaSiC_Stitched', '_Decon')
source = os.path.join(stitch_dir, f"cyc{str(dec_cycles).zfill(2)}", f"CH{str(dec_channels)}")
dest = os.path.join(decon_dir, f"cyc{str(dec_cycles).zfill(2)}", f"CH{str(dec_channels)}")
os.makedirs(dest, exist_ok=True)

subprocess.run([decon_exe, source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(ex), str(em), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_crit), str(max_block), str(GPU)])
gc.collect()
try:
    os.rename(os.path.join(source, 'deconvolved'), os.path.join(dest, 'deconvolved'))
except (FileNotFoundError):
    print("Reduce max memory.")

The following cell will compare the original to the deconvolved image.

In [ ]:
# Enter the cycle, channel, and z-plane for the image you want to evaluate.
c = 5
ch = 3
z = 8

# Crop the image by entering the x and y coordinates below.
x1 = 400
x2 = 1000
y1 = 600
y2 = 1200

source_image = imread(os.path.join(stitch_dir, f"cyc{str(c).zfill(2)}", f"CH{str(ch)}", f"{str(z).zfill(2)}.tif"))
dest = os.path.join(stitch_dir.replace('_BaSiC_Stitched', '_Decon'), f"cyc{str(c).zfill(2)}", f"CH{str(ch)}")
fig, axes = plt.subplots(1, 2, figsize=(24, 18))
im = axes[0].imshow(source_image[x1:x2,y1:y2])
fig.colorbar(im, shrink=0.5, ax=axes[0])
axes[0].set_title("Original")
decon_image = imread(os.path.join(dest, 'deconvolved', f"deconv_0000{str(z).zfill(2)}.tif"))
im = axes[1].imshow(decon_image[x1:x2,y1:y2])
fig.colorbar(im, shrink=0.5, ax=axes[1])
axes[1].set_title("Deconvolved")
fig.tight_layout()

You may now use the optimized parameters in the next notebook to process large datasets.

## 4 FIJI Clij2 plugin - EDF

The following section connects this notebook with a local installation of FIJI and a Clij2 plugin to run an Extended Depth of Focus (EDF) projection along the deconvolved image stack.  The image calculation is done on the GPU for efficient processing.  In the next cell enter the amount of memory (no more than 80% of system RAM is a good rule of thumb) to allow for the computation.  Also, enter the name of the GPU, and the parameters for the EDF calculation.  A new folder will be created for the results, and the image can be inspected in another program and in the next section.

### 4.1 EDF on GPU

In [ ]:
ij_mem = 40
edf_cycles = 1
edf_channels = 1
GPU_name = "NVIDIA RTX A4500"
radius_x = 5.0
radius_y = 5.0
sigma = 20.0

scyjava.config.add_option(f'-Xmx{str(ij_mem)}g')
ij = imagej.init('C:/Users/smith6jt/Fiji.app')

macro = """
#@ File in_folder
#@ String device
#@ File out_folder
#@ Integer radius_x
#@ Integer radius_y
#@ Integer sigma

File.openSequence(in_folder);
run("CLIJ2 Macro Extensions", "cl_device=[" + device + "]");
Ext.CLIJ_clear();
image1 = "deconvolved";
Ext.CLIJ2_push(image1);
image2 = "extended_depth_of_focus_variance_projection";
radius_x = 5.0;
radius_y = 5.0;
sigma = 20.0;
Ext.CLIJ2_extendedDepthOfFocusVarianceProjection(image1, image2, radius_x, radius_y, sigma);
Ext.CLIJ2_pull(image2);
selectImage(image1);
close();
selectImage(image2);
saveAs("Tiff", out_folder + File.separator + "EDF.tif");
"""
decon_dir = stitch_dir.replace('_BaSiC_Stitched', '_Decon')
edf_dir = decon_dir.replace('_Decon', '_EDF')
edf_source = os.path.join(decon_dir, f"cyc{str(edf_cycles).zfill(2)}", f"CH{str(edf_channels)}", "deconvolved")
edf_dest = os.path.join(edf_dir, f"cyc{str(edf_cycles).zfill(2)}", f"CH{str(edf_channels)}")
os.makedirs(edf_dest, exist_ok=True)

args ={'in_folder': edf_source, "cl_device" : GPU_name, "out_folder" : edf_dest, "radius_x" : 5.0, "radius_y" : 5.0, "sigma" : 20.0}
ij.py.run_macro(macro, args)

### 4.2 EDF comparison with deconvolution images

Visualize the results with periodic z-planes.

In [ ]:
decon_stack = sorted(glob(os.path.join(edf_source, f"deconv_0000??.tif")), key=alphanumeric_key)
decon_stack = io.imread_collection(decon_stack)
decon_image = np.asarray(decon_stack)
edf_result = imread(os.path.join(edf_dest, "EDF.tif"))

x1 = 2400
x2 = 3000
y1 = 2000
y2 = 3000

fig, axes = plt.subplots(2, 3, figsize=(18, 8))
im = axes[0,0].imshow(decon_image[6][x1:x2,y1:y2])
axes[0,0].set_title("decon_z007")
im = axes[0,1].imshow(decon_image[7][x1:x2,y1:y2])
axes[0,1].set_title("decon_z008")
im = axes[0,2].imshow(decon_image[8][x1:x2,y1:y2])
axes[0,2].set_title("decon_z009")
im = axes[1,0].imshow(decon_image[9][x1:x2,y1:y2])
axes[1,0].set_title("decon_z010")
im = axes[1,1].imshow(decon_image[10][x1:x2,y1:y2])
axes[1,1].set_title("decon_z011")
im = axes[1,2].imshow(edf_result[x1:x2,y1:y2])
axes[1,2].set_title("EDF_var_x5y5_sigma20")
fig.tight_layout()

Visualize the results as a full image.

In [ ]:
edf_result = imread(os.path.join(edf_dest, "EDF.tif"))
fig, axes = plt.subplots(figsize=(24, 18))
im = axes.imshow(edf_result)
fig.colorbar(im)
fig.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft2, fftshift
from skimage import img_as_float
from skimage.color import rgb2gray
from skimage.filters import window



x1=2000
x2=7000
y1=2000
y2=7000


# power_im1 = calculate_power_spectrum(signal_image_tiff)
# power_im_norm1 = calculate_power_spectrum(signal_image_tiff, normalize_by_mean=True)
# frequencies1, power_spectrum_1d1 = calculate_summed_power(power_im1)
# f_k1, profile1 = calculate_radial_average(signal_image_tiff, bin_size=2)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
ax = axes.ravel()
ax[0].set_title("Original image")
ax[0].imshow(signal_image_tiff, cmap='viridis', vmax=10000)
ax[1].set_title("Power Spectrum")
ax[1].imshow(np.log(power_im1), cmap='magma')
ax[2].set_title("Power Spectrum Norm")
ax[2].imshow(np.log(power_im_norm1), cmap='magma')

ax[3].set_title("Subtracted Image")
ax[3].imshow(signal_image, cmap='viridis', vmax=10000)
ax[4].set_title("Subtracted Power Spectrum")
ax[4].imshow(np.log(power_im), cmap='magma')
ax[5].set_title("Subtracted Power Spectrum Norm")
ax[5].imshow(np.log(power_im_norm), cmap='magma')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(f_k1, profile1, label='Power Spectrum')
plt.ylabel("Average Power (log scale)")
plt.yscale('log')
plt.xlabel("Frequency")
plt.title('2D Power Spectrum with Logarithmic Scale')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(frequencies1, power_spectrum_1d1, label='Power Spectrum')
plt.yscale('log')
plt.xlabel('Frequency')
plt.ylabel('Total Power (log scale)')
plt.title('1D Power Spectrum with Logarithmic Scale')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(frequencies, power_spectrum_1d, label='Subtracted Power Spectrum')
plt.yscale('log')
plt.xlabel('Frequency')
plt.ylabel('Total Power (log scale)')
plt.title('Subtracted 1D Power Spectrum with Logarithmic Scale')
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()